In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import html,dcc
from dash.dependencies import Input,Output
import dash_bootstrap_components as dbc

In [15]:
data_auto=pd.read_csv('datasets/historical_automobile_sales.csv')
data_auto

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,8/31/2023,2023,Aug,0,103.36,0.25,27852.993,1793,6,57.169,0.764155,2.6,1579.6,Executivecar,New York
524,9/30/2023,2023,Sep,0,101.55,0.07,21183.704,1028,5,59.315,0.036180,2.5,1123.4,Smallfamiliycar,California
525,10/31/2023,2023,Oct,0,124.66,0.12,15975.589,1148,9,19.472,-2.046169,2.5,1685.9,Sports,California
526,11/30/2023,2023,Nov,0,97.09,0.25,16862.288,4850,5,27.904,0.302179,2.9,2124.6,Smallfamiliycar,Georgia


In [30]:
app=dash.Dash(__name__)
year_list=[year for year in range(1980,2024)]
app.layout=html.Div(
    children=[
        html.H1(
            'Automobile Sales Statistics Dashboard',
            style={'Aligntext':'center','color':'#503D36','font-size':24}
        ),

        html.Div([
            html.Label('Select Statistics:'),

            dcc.Dropdown(
                id='dropdown-statistics',
                options=[
                     {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'},
                ],
                value='select statistics',
                placeholder='Select a report'
            )
        ]),


        html.Div([
            html.Label('Select year'),

            dcc.Dropdown(
                id='input-year',
                options=[
                    {'label':i,'value':i} for i in year_list],
                    value='Select Year',
                    style={'width': '80%', 'padding': 3, 'textAlign': 'center', 'font-size': 20}
                
            )
        ]),

        html.Div([
    html.Div(id='graph-container', className='chart-grid', style={'display': 'flex'}),
            ])
        
    ]
)
@app.callback(
    
        Output(component_id='input-year',component_property='disabled'),
    

        Input(component_id='dropdown-statistics',component_property='value')
)
def update_input_container(dropdown_statistics):
    if dropdown_statistics=='Recession Period Statistics':
        return True
    else:
        return False




@app.callback(

        Output(component_id='graph-container',component_property='children'),
    
    [
        Input(component_id='dropdown-statistics',component_property='value'),
        Input(component_id='input-year',component_property='value'),
    ],
)
def update_container(dropdown_statistics,input_year):
    recession_data=data_auto[data_auto['Recession']==1]
    yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
    average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
    exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
    unemp_rate = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()

    if dropdown_statistics=='Recession Period Statistics':
        
        R_chart1 = dcc.Graph(
                figure=px.line(yearly_rec,
                    x='Year',
                    y='Automobile_Sales',
                    title="Average Automobile Sales fluctuation over Recession Period"))
    
        R_chart2  = dcc.Graph(
                figure=px.bar(average_sales,
                    x='Vehicle_Type',
                    y='Automobile_Sales',
                    title="Average Automobile Sales by Vehicle Type during Recession Period"))
    
        R_chart3 = dcc.Graph(
                figure=px.pie(exp_rec,
                    values='Advertising_Expenditure',
                    names='Vehicle_Type',
                    title="Total Expenditures by Vehicle Type during Recession Period"))
    
        R_chart4 = dcc.Graph(
                figure=px.bar(unemp_rate,
                              x='Vehicle_Type',
                              y='Automobile_Sales',
                              title="Effects of Unemployment Rate on Automobile Sales by Vehicle Type during Recession Period"))
            
        return[
            html.Div(
              className='chart-grid',
              children=[
                  dbc.Row(
                      [dbc.Col(html.Div(children=R_chart1),
                               width={'size':10}),
                       dbc.Col(html.Div(children=R_chart2),
                               width={'size':10})]
                      )],style={'display': 'flex'}
              
            ),
            html.Div(
                className='chart-grid',
              children=[
                  dbc.Row(
                      [dbc.Col(html.Div(children=R_chart3),
                               width={'size':10}),
                       dbc.Col(html.Div(children=R_chart4),
                               width={'size':10})]
                      )],style={'display': 'flex'}
                
            )
            ]


    elif (dropdown_statistics=='Yearly Statistics' and input_year):
        yearly_data = data_auto[data_auto['Year'] == input_year]
        yas= data_auto.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas,
                x='Year',
                y='Automobile_Sales',
                title='Automobile Sales for the Year'))

# Plot 2 Total Monthly Automobile sales using line chart.
#         data['Month'] = pd.to_datetime(data['Month'])
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(mas,
                           x='Month',
                           y='Automobile_Sales',
                           title='Total Automobile Sales by Month'))

            # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart3 =dcc.Graph(figure=px.bar(avr_vdata,
                x='Year',
                y='Automobile_Sales',title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

            # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data,
                values='Advertising_Expenditure',
                names='Vehicle_Type',
                title='Total Advertising Expenditure by Vehicle Type'))




        return [
             html.Div(
              className='chart-grid',
              children=[
                  dbc.Row(
                      [dbc.Col(html.Div(children=Y_chart1),
                               width={'size':10}),
                       dbc.Col(html.Div(children=Y_chart2),
                               width={'size':10})]
                      )],style={'display': 'flex'}
              
            ),
            html.Div(
                className='chart-grid',
              children=[
                  dbc.Row(
                      [dbc.Col(html.Div(children=Y_chart3),
                               width={'size':10}),
                       dbc.Col(html.Div(children=Y_chart4),
                               width={'size':10})]
                      )],style={'display': 'flex'}
            )
        ]
        

app.run_server(debug=True,port=3004)

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,12/31/2009,2009,Dec,1,94.83,0.25,20863.845,1832,8,37.003,-0.459719,5.0,594.5,Smallfamiliycar,Illinois
109,9/30/2020,2020,Sep,1,111.62,0.07,33435.708,2622,8,17.046,-1.170773,3.7,692.8,Mediumfamilycar,Illinois
110,10/31/2020,2020,Oct,1,84.48,0.00,29408.199,4700,4,53.028,0.678547,3.6,214.0,Sports,California
111,11/30/2020,2020,Nov,1,112.39,0.07,24960.137,4945,8,24.367,-1.176222,4.2,597.1,Mediumfamilycar,New York
